## **Practicando PySpark con datos del subte de BA**

Este notebook está destinado a procesar datos vinculados a los subtes de la Ciudad de Buenos Aires, específicamente sobre la cantidad de pasajeros por molinete de todas las estaciones de la red de Subte.
Los datasets se encuentran disponible [aquí](https://data.buenosaires.gob.ar/dataset/subte-viajes-molinetes)

En el sitio [Buenos Aires Data]() se encuentra una gran variedad de datos abiertos.

### **Entorno de trabajo**
Se trabaja de manera local con una imagen de docker que incluye todo lo necesario para trabajar con PySpark

Dicha docker image está disponible [aquí](https://hub.docker.com/r/jupyter/pyspark-notebook/)

Un vistazo rápido a los datos

In [1]:
!head -2 molinetes*.csv

==> molinetes-2014.csv <==
﻿FECHA;DESDE;HASTA;LINEA;MOLINETE;ESTACION;PAX_PAGO;PAX_PASES_PAGOS;PAX_FRANQ;PAX_TOTAL
2014-04-09;09:15;09:29;B;LINEA_B_FLORIDA_E_TURN02;Florida;7;NA;NA;7

==> molinetes-2015.csv <==
periodo,fecha,desde,hasta,linea,molinete,estacion,pax_pagos,pax_pases_pagos,pax_franq,total
201501,2015-01-01,05:00:00,05:15:00,LINEA_H,LINEA_H_CASEROS_NORTE_TURN01,CASEROS,0.0,0.0,0.0,0.0

==> molinetes-2016.csv <==
﻿PERIODO;FECHA;DESDE;HASTA;LINEA;MOLINETE;ESTACION;PAX_PAGOS;PAX_PASES_PAGOS;PAX_FRANQ;TOTAL
201601;02/01/2016;05:00:00;05:15:00;LINEA_A;LINEA_A_CARABOBO_E_TURN03;CARABOBO;1;0;0;1

==> molinetes-2017.csv <==
﻿PERIODO;FECHA;DESDE;HASTA;LINEA;MOLINETE;ESTACION;PAX_PAGOS;PAX_PASES_PAGOS;PAX_FRANQ;TOTAL;ID
201701;01/01/2017;08:00:00;08:15:00;LINEA_H;LINEA_H_CASEROS_SUR_TURN02;CASEROS;1;0;0;1;1

==> molinetes-2018.csv <==
fecha,desde,hasta,linea,molinete,estacion,pax_pagos,pax_pases_pagos,pax_franq,total,periodo
2018-01-01,08:00:00,08:15:00,LineaA,LineaA_CBarros_S_Turn01

No todos los archivos mantienen la misma estructura. Un grupo tiene el caracter ',' como delimitador, mientras que para el otro es el caracter ';'.

El orden de las columnas no es el mismo para todos los archivos. Los valores para el campo *LINEA* no mantiene el mismo formato para todos los arhivos

El formato de fecha también difiere entre los archivos

### **Hands on**

In [2]:
# Para interactuar con una base de datos desde PySpark es necesario disponer del conector JDBC para dicha base de datos
PG_JDBC_PATH = "/home/jovyan/work/postgresql-42.2.12.jar"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = f"--driver-class-path {PG_JDBC_PATH} --jars {PG_JDBC_PATH} pyspark-shell"

In [4]:
spark = SparkSession.builder \
                    .master("local[*]") \
                    .appName("molinetes") \
                    .getOrCreate()

Primero veamos si hay missing/null values

In [5]:
cols_to_select=["Fecha", "Desde", "Hasta", "Linea", "Estacion"]

In [5]:
fnames = [f"molinetes-201{n}.csv" for n in [4,6,7]]
for fname in fnames:
    print(fname)
    molinetes = spark.read.csv(fname, sep=";", header=True, encoding="UTF-8")
    molinetes = molinetes.select(*cols_to_select)
    for column in molinetes.columns:
        nulls_count = molinetes.filter(molinetes[column].isNull()).count()
        print(f"Column: {column}\t Null values: {nulls_count}")
    
    molinetes = None

molinetes-2014.csv
Column: Fecha	 Null values: 0
Column: Desde	 Null values: 0
Column: Hasta	 Null values: 0
Column: Linea	 Null values: 0
Column: Estacion	 Null values: 0
molinetes-2016.csv
Column: Fecha	 Null values: 0
Column: Desde	 Null values: 0
Column: Hasta	 Null values: 0
Column: Linea	 Null values: 0
Column: Estacion	 Null values: 0
molinetes-2017.csv
Column: Fecha	 Null values: 0
Column: Desde	 Null values: 0
Column: Hasta	 Null values: 0
Column: Linea	 Null values: 0
Column: Estacion	 Null values: 0


In [6]:
fnames = [f"molinetes-201{n}.csv" for n in [5,8,9]]
for fname in fnames:
    print(fname)
    molinetes = spark.read.csv(fname, sep=",", header=True, encoding="UTF-8")
    molinetes = molinetes.select(*cols_to_select)
    for column in molinetes.columns:
        nulls_count = molinetes.filter(molinetes[column].isNull()).count()
        print(f"Column: {column}\t Null values: {nulls_count}")
    

    molinetes = None

molinetes-2015.csv
Column: Fecha	 Null values: 0
Column: Desde	 Null values: 0
Column: Hasta	 Null values: 0
Column: Linea	 Null values: 0
Column: Estacion	 Null values: 0
molinetes-2018.csv
Column: Fecha	 Null values: 78207
Column: Desde	 Null values: 78207
Column: Hasta	 Null values: 78207
Column: Linea	 Null values: 78263
Column: Estacion	 Null values: 78207
molinetes-2019.csv
Column: Fecha	 Null values: 0
Column: Desde	 Null values: 0
Column: Hasta	 Null values: 0
Column: Linea	 Null values: 0
Column: Estacion	 Null values: 0


El archivo referente al año 2018 requiere un tratamiento diferente por presentar valores nulos.

Primero, trabajaremos con los otros datasets.

In [6]:
def csv_to_df(filename, separator):
    cols_to_select = ["fecha", "desde", "linea", "estacion"]
    # Omitimos la columna Hasta dado que los registros se efectuaron cada quince minutos    
    
    molinetes = spark.read.csv(filename, encoding="UTF-8", header=True, sep=separator)
    
    # La columna referente al total de pasajeros por molinete tiene nombres diferentes entre los archivos.
    # De la siguiente forma logramos obtener el nombre de la columna que hace referencia a dichos totales
    total_column = next(filter(lambda s: 'total' in s.lower(), molinetes.columns))
    cols_to_select.append(total_column)
    molinetes = molinetes.select(*cols_to_select)
    molinetes = molinetes.withColumnRenamed(total_column, "total")
    return molinetes

def modify_columns(df):
    last_char = F.udf(lambda s: s[-1])
    df = df.withColumn("linea", last_char("linea"))
    df = df.withColumn("estacion", F.upper(F.col("estacion")))
    
    df = df.withColumnRenamed("desde", "hora")
    df = df.withColumn("hora", \
                      F.hour(df["hora"]))

    df = df.withColumn("total", F.col("total").cast("int"))
    
    # Transformar el formato del campo "Fecha" a dd/MM/yyyy en caso de ser necesario
    df = df.withColumn("fecha", \
                       F.when(F.to_date("fecha").isNotNull(), \
                              F.to_date("fecha")) \
                       .otherwise(F.to_date("fecha", "dd/MM/yyyy")))

    return df

def sort_and_group(df):
    cols = ["fecha", "hora", "linea", "estacion"]
    df = df.orderBy(cols) \
            .groupBy(cols).agg(F.sum("total").alias("total"))
    return df

In [8]:
fnames = (f"molinetes-201{n}.csv" for n in [4,6,7])
for fname in fnames:
    print(fname, end="\t")
    molinetes = csv_to_df(fname, ';')
    molinetes = modify_columns(molinetes)
    molinetes = sort_and_group(molinetes)
    molinetes.write.saveAsTable("molinetes", mode="append")
    print("saved")
    molinetes = None

molinetes-2014.csv	saved
molinetes-2016.csv	saved
molinetes-2017.csv	saved


In [9]:
fnames = (f"molinetes-201{n}.csv" for n in [5, 9])
for fname in fnames:
    print(fname, end="\t")
    molinetes = csv_to_df(fname, ',')
    molinetes = modify_columns(molinetes)
    molinetes = sort_and_group(molinetes)
    molinetes.write.saveAsTable("molinetes", mode="append")
    print("saved")
    molinetes = None

molinetes-2015.csv	saved
molinetes-2019.csv	saved


Ahora vamos a tratar los valores nulos del archivo *molinetes-18.csv*.

Cargamos el archivo en un dataframe, contamos los valores nulos y luego realizamos las transformaciones correspondientes.

In [10]:
molinetes18 = csv_to_df("molinetes-2018.csv", ',')
molinetes18.show(3)

+----------+--------+------+-------------+-----+
|     fecha|   desde| linea|     estacion|total|
+----------+--------+------+-------------+-----+
|2018-01-01|08:00:00|LineaA|Castro Barros|  1.0|
|2018-01-01|08:00:00|LineaA|         Lima|  4.0|
|2018-01-01|08:00:00|LineaA|        Pasco|  1.0|
+----------+--------+------+-------------+-----+
only showing top 3 rows



In [11]:
molinetes18 = molinetes18.dropna(subset=["Linea"])

In [30]:
for column in molinetes18.columns:
    nulls_count = molinetes18.filter(molinetes18[column].isNull()).count()
    print(f"Column: {column}\t Nulls values: {nulls_count}")

Column: fecha	 Nulls values: 0
Column: hora	 Nulls values: 0
Column: linea	 Nulls values: 0
Column: estacion	 Nulls values: 0
Column: total	 Nulls values: 0


Los valores nulos fueron eliminados y ya podemos proceder a cargar este dataframe en una tabla.

In [12]:
molinetes18 = modify_columns(molinetes18)
molinetes18 = sort_and_group(molinetes18)

In [13]:
molinetes18.show(5)

+----------+----+-----+-------------+-----+
|     fecha|hora|linea|     estacion|total|
+----------+----+-----+-------------+-----+
|2018-01-01|   8|    A|       ACOYTE|   27|
|2018-01-01|   8|    A|      ALBERTI|    9|
|2018-01-01|   8|    A|     CARABOBO|   38|
|2018-01-01|   8|    A|CASTRO BARROS|   33|
|2018-01-01|   8|    A|     CONGRESO|   36|
+----------+----+-----+-------------+-----+
only showing top 5 rows



In [14]:
molinetes18.write.saveAsTable("molinetes", mode="append")

### **Carga a una base de datos**

Ya finalizado el procesamiento y con los datos cargados en una tabla provisoria, procedemos a cargar toda la data en un base de datos en postgresql.

In [15]:
molinetes = spark.table("molinetes")
molinetes.show(10)

+----------+----+-----+-----------------+-----+
|     fecha|hora|linea|         estacion|total|
+----------+----+-----+-----------------+-----+
|2019-10-14|  21|    A|   RIO DE JANEIRO|  157|
|2019-10-14|  21|    A|   SAENZ PEÃÂ±A |   87|
|2019-10-14|  21|    A|      SAN PEDRITO|  189|
|2019-10-14|  21|    B|   ANGEL GALLARDO|  204|
|2019-10-14|  21|    B|         CALLAO.B|  281|
|2019-10-14|  21|    B|    CARLOS GARDEL|  838|
|2019-10-14|  21|    B|CARLOS PELLEGRINI|  704|
|2019-10-14|  21|    B|          DORREGO|  136|
|2019-10-14|  21|    B|       ECHEVERRIA|   51|
|2019-10-14|  21|    B| FEDERICO LACROZE|  338|
+----------+----+-----+-----------------+-----+
only showing top 10 rows



In [28]:
connection_string = "jdbc:postgresql://127.0.0.1/subte"
tablename = "public.molinetes"
connection_details = {
    "user": "<a_username>",
    "password": "<a_password>",
    "driver": "org.postgresql.Driver"
}

In [29]:
molinetes.write.jdbc(connection_string, tablename, mode="append", properties=connection_details)

Finalmente, los datos han sido cargados a una db. Asi resultan mas accesibles para crear algún dashboard de visualización